# CS 195: Natural Language Processing
## Chat and Instruct Models

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ericmanley/s26-CS195NLP/blob/main/F2_1_ChatInstruct.ipynb)

## References


* [Hugging Face Chat Basics](https://huggingface.co/docs/transformers/en/conversations)
* [SmolLM2: When Smol Goes Big — Data-Centric Training of a Small Language Model](https://arxiv.org/pdf/2502.02737)


## Demo Day

Sit with the same people you sat with last week

* Each person do a 5-min demo of **creative synthesis** project or completed **applied exploration** (or **core practice** if that's what you have)
* Write down the names of the people you presented to (you'll include this in your portfolio later)
* (optional) Nominate a cool project to show off to everyone



## Install Modules

We'll be using `transformers` version 5. You probably only need to run this if you are doing this for the first time on your own computer. If so, uncomment these two lines and run it.


In [ ]:
#import sys
#!{sys.executable} -m pip install transformers accelerate

## Large vs. Small language models

Large Language Models (GPT 5.2, Claude Opus 4.6, Grok 4.1, Gemini 3) get all the attention.

Smaller language models have come a long way too, and they require much less computation
* can often be run on a laptop or a Colab instance
* can be *fine-tuned* for specific applications with good performance


### Example: SmolLM2

Hugging Face developed a [family of small language models called SmolLM](https://huggingface.co/collections/HuggingFaceTB/smollm2) there's also a [SmolLM3](https://huggingface.co/blog/smollm3)

SmolLM2 comes in various sizes
* 135M (135 million parameters - weights in the neural network)
* 360M
* 1.7B

Contrast with the LLMs above which likely all have over 100 billion parameters and run on a cluster of devices in a data center

Each size has a **base model**, like [SmolLM2-360M](https://huggingface.co/HuggingFaceTB/SmolLM2-360M)
* *pre-trained* on lots of diverse text
* designed to predict the *next word* - it's your phone's keyboard text prediction on steroids

The *base model* is then fine-tuned on *instruction following* and *conversational data*, which make it useful for building **chat bots**.
* resulting model has a name like [SmolLM2-360M-Instruct](https://huggingface.co/HuggingFaceTB/SmolLM2-360M-Instruct)

## Building a chat bot with the `text-generation` pipeline

Setting up a chat bot works the same way as other Hugging Face models we've seen, but we'll use the `text-generation` pipeline

In [1]:
from transformers import pipeline
from accelerate import Accelerator

device = Accelerator().device

chatbot = pipeline("text-generation", model="HuggingFaceTB/SmolLM2-360M-Instruct", device = device)

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/724M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

Device set to use mps


### Chat template

*Instruct* models often allow you to pass the input in using a **Chat Template** like this

In [4]:
chat_history = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Explain gravity in one paragraph."},
]

Now lets get the response and display what is returned

In [6]:
response = chatbot(chat_history)
response

[{'generated_text': [{'role': 'system',
    'content': 'You are a helpful assistant.'},
   {'role': 'user', 'content': 'Explain gravity in one paragraph.'},
   {'role': 'assistant',
    'content': "Gravity is a fundamental force of nature that attracts all objects with mass towards each other. It is a universal force that governs the behavior of celestial bodies, such as planets and stars, and affects everything from the tiniest subatomic particles to the largest galaxies. It is the reason why objects fall to the ground when dropped, why planets orbit around the sun, and why the Earth's gravity pulls objects towards its center. Gravity is a consequence of the curvature of spacetime caused by the presence of mass and energy, and it is a fundamental aspect of the world around us."}]}]

**Notice:**
* it just uses normal lists and dictionaries
* it returns the entire chat history on the `'generated_text'` key

If you wanted to just print out the response, you could do it like this

In [9]:
print("ASSISTANT:", response[0]['generated_text'][-1]['content'])

ASSISTANT: Gravity is a fundamental force of nature that attracts all objects with mass towards each other. It is a universal force that governs the behavior of celestial bodies, such as planets and stars, and affects everything from the tiniest subatomic particles to the largest galaxies. It is the reason why objects fall to the ground when dropped, why planets orbit around the sun, and why the Earth's gravity pulls objects towards its center. Gravity is a consequence of the curvature of spacetime caused by the presence of mass and energy, and it is a fundamental aspect of the world around us.


## Multi-turn conversations

If you want to have a multi-turn conversation, you need to pass *the entire chat history* to the model - it doesn't have any inherent memory of the output it just gave you

In [11]:
another_chat_history = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Explain gravity in one paragraph."},
    {"role": "assistant", "content": "Gravity is a fundamental force of nature that causes objects with mass to attract each other. According to Sir Isaac Newton, the force of gravity between two objects depends on their masses and the distance between them. The larger the mass of the objects, the greater the gravitational pull. This force is responsible for keeping planets in orbit around the sun and causes objects to fall towards the ground when dropped. Albert Einstein also described gravity as a curvature of spacetime caused by massive objects, which in turn warps the fabric of spacetime around them."},
    {"role": "user", "content": "Which of those two do you think has had a bigger impact on the field?"}
]

next_response = chatbot(another_chat_history)
print(next_response[0]['generated_text'][-1]['content'])

According to various studies, Newton's law of universal gravitation, which describes the force of gravity between two objects, has had a more significant impact on the field of physics. This is because it provided a precise and consistent description of gravity, which has been widely accepted and has been used to explain many phenomena in physics, such as the motion of planets, the behavior of comets, and the structure of the universe. While Einstein's theory of general relativity has been incredibly successful in predicting phenomena, such as gravitational waves and the bending of light around massive objects, Newton's law remains a cornerstone of modern physics and has been widely recognized for its simplicity, elegance, and accuracy.


## Exercise

Write a loop that allows for back-and-forth conversation with the model. Make sure to keep track of the full history of the chat as you go.

## Evaluating Chat Models: Benchmarks

A benchmark is a dataset with one or more reference answers that can be used to measure a model's response (like the reference summaries we compared against with ROUGE)


Model benchmarking is a big deal - companies like to report how well their models perform on all kinds of benchmarks

For example, see the **performance** tab here: https://deepmind.google/models/gemini/pro/

Take a look at this benchmark for multi-turn conversations: https://huggingface.co/datasets/HuggingFaceH4/mt_bench_prompts

**Group Discussion:** What are some things you notice about this data?



## Evaluating Chat Models: Human Evaluators

Language models are often evaluated by having humans perform A/B testing where two models respond to the same prompt, and the human indicates which was better.

Try it out here: https://arena.ai

## Group Exercise

Do the following as a group:
* Come up with 5 language model prompts - what are some questions/instructions you think would help you decide how good a language model is?
* Test them using [SmolLM2-360M-Instruct](https://huggingface.co/HuggingFaceTB/SmolLM2-360M-Instruct) and [Qwen/Qwen2.5-0.5B-Instruct](https://huggingface.co/Qwen/Qwen2.5-0.5B-Instruct)
* Have each person in your group vote on which one they thought was the best
* Write down the results

## Applied Exploration

Choose two instruct models of similar size: https://huggingface.co/models?pipeline_tag=text-generation&sort=trending&search=instruct
  * Link to the model cards for the models you're using and describe each of them

Do one of the following:

1. Go to https://huggingface.co/datasets and find a dataset suitable to use as a benchmark, and compare the performance of the two models. It doesn't have to be a conversational benchmark - it could be a text classification, summarization, math, etc. dataset, as long as you can instruct the model to answer it. And, you don't have to use the whole dataset.
    * link to and describe the dataset
    * describe how you compared the performance (e.g., what metric did you use?)
    * report the results

OR

2. Come up with your own fun benchmark (Taylor Swift trivia, AI Dungeon Master, Joke telling, etc.), generate responses for both models, and have another person rate the answers.
    * Describe what you did
    * report the results
